<a href="https://colab.research.google.com/github/DHARAJAK/Hand-on-Large-Language-Models/blob/main/hands_on_llm_chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
                                             device_map = "cuda",
                                             torch_dtype = "auto",
                                             trust_remote_code = True)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Create a pipeline
pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 500,
    do_sample = False,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda


In [3]:
#Prompt
messages = [
    {"role": "user","content":"Create a funny joke about chickens."}
]


# Generate the output
output = pipe(messages)
print(output[0]["generated_text"])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


[{'role': 'user', 'content': 'Create a funny joke about chickens.'}, {'role': 'assistant', 'content': ' Why did the chicken join the band? Because it had the drumsticks!'}]


In [6]:
# Apply prompt template
prompt = pipe.tokenizer.apply_chat_template(messages,tokenize = False)
print(prompt)

<|user|>
Create a funny joke about chickens.<|end|>
<|endoftext|>


In [10]:
#Using a high temperature
output = pipe(messages, do_sample = True, temperature = 1)
print(output[0]["generated_text"])

[{'role': 'user', 'content': 'Create a funny joke about chickens.'}, {'role': 'assistant', 'content': ' Why did the chicken join a band? Because it wanted to quail!'}]


In [12]:
#Using a high top_p
output = pipe(messages, do_sample = True, top_p = 1)
print(output[0]["generated_text"])

[{'role': 'user', 'content': 'Create a funny joke about chickens.'}, {'role': 'assistant', 'content': ' Why did the farmer bring a bar of soap to the farm?\n\n\nBecause he wanted to get rid of the rooster’s “dirty politics” and make the farm’s “egg economics” more “egg-stra”!'}]


In [13]:
# Prompt components
persona = "You are an expert in Large Language models. You excell at breaking down complex papers into digestible summaries.\n"
instruction = "Summarize the key findings of the paper provided \n"
context = "Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper. \n"
data_format = "Create a bullet point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results. \n"
audience = "The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models \n"
tone = "The tone should be professional and clear. \n"
text = "MY TEXT TO SUMMARIZE"
data = f"Text to summarize: {text}"

# The full prompt- remove and add pieces to view its impact on the generated output
query = persona + instruction + context + data_format + audience + tone + data

In [14]:
output = pipe(query, do_sample = False)
print(output[0]["generated_text"])

You are an expert in Large Language models. You excell at breaking down complex papers into digestible summaries.
Summarize the key findings of the paper provided 
Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper. 
Create a bullet point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results. 
The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models 
The tone should be professional and clear. 
Text to summarize: MY TEXT TO SUMMARIZE

Title: "Exploring the Impact of Large Language Models on Natural Language Processing"

Abstract:

The advent of Large Language Models (LLMs) has revolutionized the field of Natural Language Processing (NLP). This paper explores the impact of LLMs on various NLP tasks, including text generation, translation, and sentiment analysis. We conducted a comprehensive r

In [15]:
# Use a single example of using the made-up word in a sentence
one_shot_prompt = [
    {
        "role": "user",
        "content": "A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:"
    },
    {
        "role": "assistant",
        "content": "I have a Gigamuru that my uncle gave me as a gift. I love to play it at home."
    },
    {
        "role": "user",
        "content": "To 'screeg something is to swing a sword at it. An example of a setence that uses the word screeg is:"
    }
]

print(tokenizer.apply_chat_template(one_shot_prompt, tokenize= False))

<|user|>
A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:<|end|>
<|assistant|>
I have a Gigamuru that my uncle gave me as a gift. I love to play it at home.<|end|>
<|user|>
To 'screeg something is to swing a sword at it. An example of a setence that uses the word screeg is:<|end|>
<|endoftext|>


In [17]:
# Generate the output
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

[{'role': 'user', 'content': "A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:"}, {'role': 'assistant', 'content': 'I have a Gigamuru that my uncle gave me as a gift. I love to play it at home.'}, {'role': 'user', 'content': "To 'screeg something is to swing a sword at it. An example of a setence that uses the word screeg is:"}, {'role': 'assistant', 'content': ' During the medieval tournament, the knight skillfully screeg the wooden target, impressing the onlookers with his precision and strength.'}]


In [24]:
outputs[0]

{'generated_text': [{'role': 'user',
   'content': "A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:"},
  {'role': 'assistant',
   'content': 'I have a Gigamuru that my uncle gave me as a gift. I love to play it at home.'},
  {'role': 'user',
   'content': "To 'screeg something is to swing a sword at it. An example of a setence that uses the word screeg is:"},
  {'role': 'assistant',
   'content': ' During the medieval tournament, the knight skillfully screeg the wooden target, impressing the onlookers with his precision and strength.'}]}

In [26]:
outputs[0]["generated_text"][3]['content']

' During the medieval tournament, the knight skillfully screeg the wooden target, impressing the onlookers with his precision and strength.'

In [36]:
# Create name and slogan for a product
product_prompt = [
    {"role":"user","content": "Create a name and slogan for a chatbot that leverages LLMs."}
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"][1]['content']
print(product_description)

 Name: ChatSage
Slogan: "Unleashing the power of AI to enhance your conversations."


In [38]:
# Based on a name and slogan for a product, generate a sales pitch
sales_prompt = [
    {
        "role": "user",
        "content": f"Create a verty short sales pitch for the following product: {product_description}"
    }
]

outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"][1]['content']
print(sales_pitch)

 Introducing ChatSage, the revolutionary AI-powered tool designed to elevate your conversations to new heights. With our cutting-edge technology, we unleash the power of AI to enhance your interactions, making them more engaging, insightful, and meaningful. Say goodbye to mundane chats and hello to a world of endless possibilities with ChatSage. Experience the future of communication today!


In [42]:
# Answering with chain-of-thought
cot_prompt = [
    {"role":"user", "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls.How many tennis balls does he have now?"},
    {"role":"assistant", "content": "Roger started with 5 balls.2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."},
    {"role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"}
]

outputs = pipe(cot_prompt)
print(outputs[0]["generated_text"][3]['content'])

 The cafeteria started with 23 apples. They used 20 apples to make lunch, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they now have 3 + 6 = 9 apples. The answer is 9.


In [45]:
# Zero-shot chain-of-thought
zeroshot_cot_prompt = [{
    "role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more , how many apples do they have? Let's think step-by-step"
}]

# Generate the output
outputs = pipe(zeroshot_cot_prompt)


In [48]:
print(outputs[0]["generated_text"][1]['content'])

 Step 1: The cafeteria starts with 23 apples.

Step 2: They used 20 apples to make lunch. To find out how many apples are left, we subtract the number of apples used from the initial number of apples:
23 apples - 20 apples = 3 apples

Step 3: The cafeteria bought 6 more apples. To find out the total number of apples they have now, we add the number of apples bought to the remaining apples:
3 apples + 6 apples = 9 apples

So, the cafeteria has 9 apples now.


In [49]:
# Zero-shot tree-of-thought
zeroshot_tot_prompt = [
    {"role": "user", "content": "Imagine three different experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point then they leave. The question is 'The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?' Make sure to discuss the results ."}
]

outputs = pipe(zeroshot_tot_prompt)
print(outputs[0]["generated_text"][1]['content'])

 Expert 1:
Step 1: Start with the initial number of apples, which is 23.

Expert 2:
Step 1: Subtract the number of apples used for lunch, which is 20.

Expert 3:
Step 1: Add the number of apples bought, which is 6.

Expert 1:
Step 2: Calculate the remaining apples after lunch: 23 - 20 = 3 apples.

Expert 2:
Step 2: Calculate the final number of apples after buying more: 3 + 6 = 9 apples.

Expert 3:
Step 2: Calculate the final number of apples after buying more: 3 + 6 = 9 apples.

All experts agree on the final result: The cafeteria has 9 apples.
